# Notebook setup

In [1]:
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
import pydotplus
from sklearn.feature_selection import mutual_info_classif

# Please set this to the path to you graphviz
os.environ['PATH'] = "C:/Program Files (x86)/Graphviz/bin"

# Feature Selection

In [21]:
# Reading the raw_combined.csv
df_raw = pd.read_csv('raw_combined.csv')

# Reading the feature list metadata
df_feat_info = pd.read_csv('feature_info.csv', sep = '|')

# Selecting features to train the model on 
selected_feature = df_feat_info[df_feat_info['category'].isin(['Demographic','Education'])]['id'].tolist()

# Uncomment the feature that you are trying to predit
pred_feat = 'SALARY'
# pred_feat = 'NOCPR'

# Binning Salary
bins = [0,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,110000,120000,130000,140000,150000,160000,170000,180000,190000,200000,210000,220000,230000,240000,250000,260000,270000,280000,290000,300000,310000,320000,330000,340000,350000]
labels = [0,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,110000,120000,130000,140000,150000,160000,170000,180000,190000,200000,210000,220000,230000,240000,250000,260000,270000,280000,290000,300000,310000,320000,330000,340000]
df_raw['SALARY_BIN'] = pd.cut(df_raw['SALARY'], bins=bins, labels=labels, right=False)
df_raw['SALARY_BIN'] = df_raw['SALARY_BIN'].astype(float)
pred_feat = 'SALARY_BIN'
# Filtering the raw file
df_features = df_raw[selected_feature]
df_pred = df_raw[pred_feat]

C:\Users\shrey\AppData\Local\Temp\ipykernel_27972\2975606345.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('raw_combined.csv')


In [12]:
# Filling in -1 for na
df_features = df_features.fillna(-1)
df_pred = df_pred.fillna(-1)

# This step is very cpu intensive and slow, it can take time to run
mi = mutual_info_classif(df_features, df_pred, n_jobs = -1)

df_mi = pd.DataFrame.from_dict(dict(zip(df_features.columns.tolist(), mi.tolist())), orient='index').reset_index()
df_mi.columns = ['Col_name','MI']

# Setting the MI threshold
mi_thres = 0.05
filtered_mi_col = df_mi[df_mi['MI'] > mi_thres]['Col_name'].tolist()

df_features = df_features[filtered_mi_col]